In [1]:
import pandas as pd
import numpy as np
import math
import datetime

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)


def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

def convert(minutes): 
    hour, minu = divmod(minutes, 60) 
    return "%d:%02d" % (hour, minu) 

In [17]:
"""gained_time1=0
to_2=[]
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i, i+1)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time1 :
            gained_time1+=(diff-ctime)
            print(f"{day}: primo : {paese} : {gained_time1}")
        else:
            gained_time1+=diff
            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")
            to_2.append(i)
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")"""

'gained_time1=0\nto_2=[]\ndays = [\'martedi\', \'mercoledi\',\'giovedì\']\n#for day in days:\nday=\'martedi\'\nfor i in range(len(time_table[day].dropna())-2):\n    try:\n        diff=difference(day, i, i+1)\n        row=dft[get_row(dft, table[day][i]).values]\n        paese=table[day][i+1]\n        value=row.loc[:,get_column(dft, paese).values]\n        ctime=float(value.values[0][0])\n        #gained_time1=gained_time1+(diff-float(value.values[0][0]))\n        #gained_time2=gained_time2+(diff-float(value.values[0][0]))\n        if (diff+(diff/20))>ctime-gained_time1 :\n            gained_time1+=(diff-ctime)\n            print(f"{day}: primo : {paese} : {gained_time1}")\n        else:\n            gained_time1+=diff\n            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")\n            to_2.append(i)\n    except:\n        print(value)\n    #if diff<dft[row[paese].values]:\n    #    print("Not acceptable!")'

In [18]:
"""impossible=[]
gained_time2=0
for i in range(len(to_2)-1):
    try:
        curr=to_2[i]
        diff=difference(day, curr, to_2[i+1])
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][to_2[i+1]]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time2 :
            gained_time2+=(diff-ctime)
            print(f"{day}: secondo : {paese} : {gained_time2}")
        else:
            print(f"{day}: impossible : {paese} : {diff}  {ctime}")
            impossible.append(i)
    except:
        print(f"-----{value}")"""

'impossible=[]\ngained_time2=0\nfor i in range(len(to_2)-1):\n    try:\n        curr=to_2[i]\n        diff=difference(day, curr, to_2[i+1])\n        row=dft[get_row(dft, table[day][i]).values]\n        paese=table[day][to_2[i+1]]\n        value=row.loc[:,get_column(dft, paese).values]\n        ctime=float(value.values[0][0])\n        #gained_time1=gained_time1+(diff-float(value.values[0][0]))\n        #gained_time2=gained_time2+(diff-float(value.values[0][0]))\n        if (diff+(diff/20))>ctime-gained_time2 :\n            gained_time2+=(diff-ctime)\n            print(f"{day}: secondo : {paese} : {gained_time2}")\n        else:\n            print(f"{day}: impossible : {paese} : {diff}  {ctime}")\n            impossible.append(i)\n    except:\n        print(f"-----{value}")'

In [2]:
data={}
to_write1 = pd.DataFrame(data)
to_write2 = pd.DataFrame(data)
to_write1["lunedi"] = table["lunedi"][:].dropna()

In [3]:
#FINAL ALGORITHM (TUE, WED, THU) to extract route
days = ["martedi", "mercoledi", "giovedi"]

for day in days:
    start = dft[get_row(dft, 'Cremona, sesto 39').values]
    #day='martedi'
    #Liste dei due furgoni
    f1 = []
    f2 = []
    #indici
    i1 = -1
    i2 = -1
    #Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
    row1 = start
    row2 = start
    #Lista dei paesi non raggiungibili
    impossibile = []
    #
    for i in range(len(time_table[day].dropna())):
        paese=table[day][i]
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value1=row1.loc[:,get_column(dft, paese).values]
        value2=row2.loc[:,get_column(dft, paese).values]
        #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
        if(i1 != -1 and i2 != -1):
            diff1=difference(day, i1, i)
            diff2=difference(day, i2, i)
            #print(f"{diff1}, {diff2}")
        else:
            #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
            if(i1 == -1):
                diff1 = value1.values[0][0]
            if(i2 == -1):
                diff2 = value2.values[0][0]

        if(diff1 == 0):
            f2.append(i)
            i2 = i
            row2 = dft[get_row(dft, table[day][i]).values]
            continue
        if(diff2 == 0):
            f1.append(i)
            i1 = i
            row1 = dft[get_row(dft, table[day][i]).values]
            continue

        #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5 minuti) si sceglie il migliore

        if(float(value1.values[0][0])<= float(diff1) + 15 or float(value2.values[0][0])<= float(diff2) + 15):
            if(value1.values[0][0] <= value2.values[0][0]):
                f1.append(i)
                i1 = i
                """print(i)
                if(i >= 13):
                    print(f"{diff1}, {diff2}")"""
                row1 = dft[get_row(dft, table[day][i]).values]
            else:
                f2.append(i)
                i2 = i
                row2 = dft[get_row(dft, table[day][i]).values]
        #Impossibilità di raggiungere la locazione in tempo
        else:
            print(f"current time1 {diff1} time {float(value1.values[0][0])} , current time2 {diff2} time {float(value2.values[0][0])}")
            impossibile.append(i)

    """print(f1)
    print(f2)"""
    cp = [p for p in table[day][f1].dropna()]
    to_write1[day]= pd.Series(cp)
    cp = [p for p in table[day][f2].dropna()]
    to_write2[day]= pd.Series(cp)

In [4]:
to_write1["venerdi"] = table["venerdi"][:].dropna()

In [5]:
to_write1

,lunedi,martedi,mercoledi,giovedi,venerdi
0,sesto ed,MANERBIO,Paderno,manerbio,Capergnanica
1,grumello,"Cremona, Ugolani Dati, 4",San Bassano,"Cremona, Ugolani Dati, 4",monte cremasco
2,pizzighettone,Bonemerse,Castelleone,"Cremona, Ugolani Dati",spino d'adda
3,MONTODINE,Sospiro,Spino d'adda,grontardo,pandino
4,ripalta cremasca,Vescovato,Romanengo,pescarolo,casaletto
5,CAPRALBA,Piadena,Pandino,vescovato,bagnolo
6,CASALE CREMASCO,Gussola,Palazzo Pignano,cingia,"crema, civerchi"
7,CAMISANO,Solarolo,Rivolta D'adda,solarolo,vaiano
8,TRIGOLO,"Scandolara Ravara, Italia",Crema,ostiano,trescore
9,CASTELLEONE,Casalmaggiore,Madignano,martignana,rivolta d'adda


In [6]:
#FINAL ALGORITHM (MON, FRI) to extract route
start = dft[get_row(dft, 'Cremona, sesto 39').values]

days = ["lunedi", "venerdi"]
for day in days:
    #Liste dei due furgoni
    f1 = []

    #indici
    i1 = -1

    #Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
    row1 = start

    #Lista dei paesi non raggiungibili
    impossibile = []
    for i in range(len(time_table[day].dropna())-1):
        paese=table[day][i]
        #print(paese)
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value1=row1.loc[:,get_column(dft, paese).values]

        #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
        if(i1 != -1):
            diff1=difference(day, i1, i)
            #print(f"{diff1}, {diff2}")
        else:
            #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
            if(i1 == -1):
                diff1 = value1.values[0][0]

        #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5 minuti) si sceglie il migliore
        #print(f"diff: {diff1} , value: {value1.values[0][0]}")
        if(float(value1.values[0][0])<= float(diff1)+5):

            f1.append(i)
            i1 = i
            row1 = dft[get_row(dft, table[day][i]).values]

        #Impossibilità di raggiungere la locazione in tempo
        else:
            print(f"current time1 {diff1} time {float(value1.values[0][0])}")
            impossibile.append(i)
    copia = [p for p in table[day][f1]]
    to_write1[day]=pd.Series(copia)

In [7]:
to_write1

,lunedi,martedi,mercoledi,giovedi,venerdi
0,sesto ed,MANERBIO,Paderno,manerbio,Capergnanica
1,grumello,"Cremona, Ugolani Dati, 4",San Bassano,"Cremona, Ugolani Dati, 4",monte cremasco
2,pizzighettone,Bonemerse,Castelleone,"Cremona, Ugolani Dati",spino d'adda
3,MONTODINE,Sospiro,Spino d'adda,grontardo,pandino
4,ripalta cremasca,Vescovato,Romanengo,pescarolo,casaletto
5,CAPRALBA,Piadena,Pandino,vescovato,bagnolo
6,CASALE CREMASCO,Gussola,Palazzo Pignano,cingia,"crema, civerchi"
7,CAMISANO,Solarolo,Rivolta D'adda,solarolo,vaiano
8,TRIGOLO,"Scandolara Ravara, Italia",Crema,ostiano,trescore
9,CASTELLEONE,Casalmaggiore,Madignano,martignana,rivolta d'adda


In [8]:
to_write2

,martedi,mercoledi,giovedi
0,"Cremona, Felice Cavallotti, 2","Cremona, Palosca","Cremona, sesto"
1,Casalpusterlengo,Casalbuttano,pizzighettone
2,"Crema, Civerchi",Castelverde,casalpusterlengo
3,Vailate,NaN,"pizzighettone, giuseppe"
4,Pianengo,NaN,izano
5,Offanengo,NaN,offanengo
6,Ostiano,NaN,"Crema, Civerchi, 9"
7,Persico Dosimo,NaN,casalmaggiore
8,Gadesco,NaN,NaN
9,Pescarolo,NaN,NaN


In [9]:
#to_write1=to_write1[['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']]
to_write1.to_csv("../Dati/turni_f/f1_original.csv", index=False, sep=";", encoding = "ISO-8859-1")

In [10]:
to_write2.to_csv("../Dati/turni_f/f2_original.csv", index=False, sep=";", encoding = "ISO-8859-1")

In [11]:
#FINAL GREEDY ALGORITHM (MON, FRI)
#Single pickup
#paese corrente
curr = 'Cremona, sesto 39'
h = 60*8
#riga distanze corrente
row = dft[get_row(dft, curr).values]
sequence = []
seq_h = []
day = 'lunedi'
for x in range(len(table[day].dropna())):
    #lista delle destinazioni e dei rispettivi tempi di arrivo
    dest = []
    values = []
    for i in range(len(table[day].dropna())):
        #controllo che non sia già stata visitata
        #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata
        if(table[day][i] not in sequence):
            dest.append(table[day][i])
            paese = table[day][i]
            values.append(row.loc[:,get_column(dft, paese).values].values[0])
    #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
    curr = dest[values.index(min(values))]
    h = h + math.ceil(min(values))
    seq_h.append(convert(h))
    #seq_h.append(math.ceil(min(values)))
    sequence.append(curr)
    #aggiorno la riga
    row = dft[get_row(dft, curr).values]
    
output=zip(sequence, seq_h)
for el in output:
    print(el)

('sesto ed', '8:11')
('grumello', '8:17')
('SORESINA', '8:29')
('TRIGOLO', '8:38')
('CASTELLEONE', '8:50')
('MONTODINE', '8:57')
('ripalta cremasca', '9:02')
('RICENGO', '9:14')
('SERGNANO', '9:21')
('CASALE CREMASCO', '9:26')
('CAMISANO', '9:32')
('CAPRALBA', '9:48')
('SONCINO', '10:09')
('pizzighettone', '10:40')


In [12]:
#FINAL GREEDY ALGORITHM (TUE, WED, THU)
#Double
#paese corrente
curr = 'Cremona, sesto 39'
h1 = 60*8
h2= 60*8
#riga distanze corrente
row1 = dft[get_row(dft, curr).values]
row2 = dft[get_row(dft, curr).values]
sequence1 = []
seq_h1 = []
sequence2 = []
seq_h2 = []
day = 'giovedi'
for x in range(len(table[day].dropna())):
    #lista delle destinazioni e dei rispettivi tempi di arrivo
    dest1 = []
    values1 = []
    dest2 = []
    values2 = []
    valuesa = []
    valuesb = []
    for i in range(len(table[day].dropna())):
        #controllo che non sia già stata visitata
        #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata     
        if(table[day][i] not in sequence1 and table[day][i] not in sequence2):
            dest1.append(table[day][i])
            paese = table[day][i]
            valuesa.append(row1.loc[:,get_column(dft, paese).values].values[0])
            dest2.append(table[day][i])
            paese = table[day][i]
            valuesb.append(row2.loc[:,get_column(dft, paese).values].values[0])
    #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
    for k in valuesa:
        values1.append(k[np.argmin(k)])
    for k in valuesb:
        values2.append(k[np.argmin(k)])
    try:
        if( min(values1)>=min(values2) ):
            curr1 = dest1[np.argmin(values1)]
            h1 = h1 + math.ceil(values1[np.argmin(values1)])
            seq_h1.append(convert(h1))
            #seq_h.append(math.ceil(min(values)))
            sequence1.append(curr1)
            #aggiorno la riga
            row1 = dft[get_row(dft, curr1).values]
        else:
            curr2 = dest2[np.argmin(values2)]
            h2 = h2 + math.ceil(values2[np.argmin(values2)])
            seq_h2.append(convert(h2))
            #seq_h.append(math.ceil(min(values)))
            sequence2.append(curr2)
            #aggiorno la riga
            row2 = dft[get_row(dft, curr2).values]
    except:
        print(sequence1)
        print(sequence2)
    
output1=zip(sequence1, seq_h1)
output2=zip(sequence2, seq_h2)
print("Furgoncino1:")
for el in output1:
    print(el)
print("\nFurgoncino2:")
for el in output2:
    print(el)

Furgoncino1:
('Cremona, sesto', '8:01')
('Cremona, Ugolani Dati, 4', '8:08')
('persico dosimo', '8:19')
('grontardo', '8:28')
('vescovato', '8:36')
('ostiano', '8:48')
('isola dovarese', '9:01')
('piadena', '9:12')
('san giovanni', '9:20')
('casalmaggiore', '9:33')
('manerbio', '10:28')
('izano', '11:02')
('offanengo', '11:09')
('Crema, Civerchi, 9', '11:21')
('casalpusterlengo', '11:51')
('pizzighettone, giuseppe', '12:05')

Furgoncino2:
('Cremona, Ugolani Dati', '8:07')
('gadesco', '8:19')
('pescarolo', '8:30')
('cingia', '8:46')
('solarolo', '8:52')
('martignana', '9:02')
('pizzighettone', '9:56')


In [13]:
#TIME routing
data={}
dffurgoncino1=pd.DataFrame(data)
dffurgoncino2=pd.DataFrame(data)
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi'] 

start = dft[get_row(dft, 'Cremona, sesto 39').values]
for day in week:
    if(day=='lunedi' or day=='venerdi'):
        #FINAL ALGORITHM (MON, FRI)
        paesi = []
        curr = 'Cremona, sesto 39'
        h = 60*8
        #riga distanze corrente
        row = dft[get_row(dft, curr).values]
        sequence = []
        seq_h = []
        for x in range(len(table[day].dropna())):
            #lista delle destinazioni e dei rispettivi tempi di arrivo
            dest = []
            values = []
            for i in range(len(table[day].dropna())):
                #controllo che non sia già stata visitata
                #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata
                if(table[day][i] not in sequence):
                    dest.append(table[day][i])
                    paese = table[day][i]
                    values.append(row.loc[:,get_column(dft, paese).values].values[0])
            #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
            curr = dest[values.index(min(values))]
            h = h + math.ceil(min(values))
            seq_h.append(convert(h))
            #seq_h.append(math.ceil(min(values)))
            sequence.append(curr)
            paesi.append(curr)
            #aggiorno la riga
            row = dft[get_row(dft, curr).values]
        dffurgoncino1[day]=pd.Series(paesi)
    else:
        #FINAL GREEDY ALGORITHM (TUE, WED, THU)
        paesi1 = []
        paesi2 = []
        curr = 'Cremona, sesto 39'
        h1 = 60*8
        h2= 60*8
        #riga distanze corrente
        row1 = dft[get_row(dft, curr).values]
        row2 = dft[get_row(dft, curr).values]
        sequence1 = []
        seq_h1 = []
        sequence2 = []
        seq_h2 = []
        for x in range(len(table[day].dropna())):
            #lista delle destinazioni e dei rispettivi tempi di arrivo
            dest1 = []
            values1 = []
            dest2 = []
            values2 = []
            valuesa = []
            valuesb = []
            for i in range(len(table[day].dropna())):
                #controllo che non sia già stata visitata
                #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata     
                if(table[day][i] not in sequence1 and table[day][i] not in sequence2):
                    dest1.append(table[day][i])
                    paese = table[day][i]
                    valuesa.append(row1.loc[:,get_column(dft, paese).values].values[0])
                    dest2.append(table[day][i])
                    paese = table[day][i]
                    valuesb.append(row2.loc[:,get_column(dft, paese).values].values[0])
            #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
            for k in valuesa:
                values1.append(k[np.argmin(k)])
            for k in valuesb:
                values2.append(k[np.argmin(k)])
            try:
                if( min(values1)>=min(values2) ):
                    curr1 = dest1[np.argmin(values1)]
                    h1 = h1 + math.ceil(values1[np.argmin(values1)])
                    seq_h1.append(convert(h1))
                    #seq_h.append(math.ceil(min(values)))
                    sequence1.append(curr1)
                    paesi1.append(curr1)
                    #aggiorno la riga
                    row1 = dft[get_row(dft, curr1).values]
                else:
                    curr2 = dest2[np.argmin(values2)]
                    h2 = h2 + math.ceil(values2[np.argmin(values2)])
                    seq_h2.append(convert(h2))
                    #seq_h.append(math.ceil(min(values)))
                    sequence2.append(curr2)
                    paesi2.append(curr2)
                    #aggiorno la riga
                    row2 = dft[get_row(dft, curr2).values]
            except:
                print(sequence1)
                print(sequence2)
        #dffurgoncino1[day]=table[day][f1].reset_index(drop=True)
        #dffurgoncino2[day]=table[day][f2].reset_index(drop=True)
        dffurgoncino1[day]=pd.Series(paesi1)
        dffurgoncino2[day]=pd.Series(paesi2)

In [14]:
dffurgoncino1

,lunedi,martedi,mercoledi,giovedi,venerdi
0,sesto ed,"Cremona, Ugolani Dati, 4",Castelverde,"Cremona, sesto","crema, civerchi"
1,grumello,Persico Dosimo,Annicco,"Cremona, Ugolani Dati, 4",Capergnanica
2,SORESINA,Ostiano,Castelleone,persico dosimo,bagnolo
3,TRIGOLO,San giovanni,Madignano,grontardo,vaiano
4,CASTELLEONE,"Scandolara Ravara, Italia",Crema,vescovato,monte cremasco
5,MONTODINE,Casalmaggiore,Palazzo Pignano,ostiano,dovera
6,ripalta cremasca,Casalpusterlengo,Pandino,isola dovarese,pandino
7,RICENGO,"Crema, Civerchi",Rivolta D'adda,piadena,agnadello
8,SERGNANO,Vailate,Spino d'adda,san giovanni,rivolta d'adda
9,CASALE CREMASCO,NaN,Romanengo,casalmaggiore,spino d'adda


In [15]:
dffurgoncino1.to_csv('../Dati/turni_f/furgoncino1_routing.csv', index=False, sep=";", encoding = "ISO-8859-1")

In [16]:
dffurgoncino2.to_csv('../Dati/turni_f/furgoncino2_routing.csv', index=False, sep=";", encoding = "ISO-8859-1")

In [17]:
#KM distances routing
data={}
dffurgoncino1_km=pd.DataFrame(data)
dffurgoncino2_km=pd.DataFrame(data)
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi'] 

start = dfd[get_row(dft, 'Cremona, sesto 39').values]
for day in week:
    if(day=='lunedi' or day=='venerdi'):
        #FINAL ALGORITHM (MON, FRI)
        paesi = []
        curr = 'Cremona, sesto 39'
        d = 0
        #riga distanze corrente
        row = dfd[get_row(dfd, curr).values]
        sequence = []
        seq_d = []
        for x in range(len(table[day].dropna())):
            #lista delle destinazioni e dei rispettive distanze
            dest = []
            values = []
            for i in range(len(table[day].dropna())):
                #controllo che non sia già stata visitata
                #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata
                if(table[day][i] not in sequence):
                    dest.append(table[day][i])
                    paese = table[day][i]
                    values.append(row.loc[:,get_column(dfd, paese).values].values[0])
            #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
            curr = dest[values.index(min(values))]
            d = d + min(values)
            seq_d.append(d)
            #seq_h.append(math.ceil(min(values)))
            sequence.append(curr)
            paesi.append(curr)
            #aggiorno la riga
            row = dfd[get_row(dfd, curr).values]
        dffurgoncino1_km[day]=pd.Series(paesi)
    else:
        #FINAL GREEDY ALGORITHM (TUE, WED, THU)
        paesi1 = []
        paesi2 = []
        curr = 'Cremona, sesto 39'
        d1 = 0
        d2 = 0
        #riga distanze corrente
        row1 = dfd[get_row(dfd, curr).values]
        row2 = dfd[get_row(dfd, curr).values]
        sequence1 = []
        seq_d1 = []
        sequence2 = []
        seq_d2 = []
        for x in range(len(table[day].dropna())):
            #lista delle destinazioni e dei rispettivi tempi di arrivo
            dest1 = []
            values1 = []
            dest2 = []
            values2 = []
            valuesa = []
            valuesb = []
            for i in range(len(table[day].dropna())):
                #controllo che non sia già stata visitata
                #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata     
                if(table[day][i] not in sequence1 and table[day][i] not in sequence2):
                    dest1.append(table[day][i])
                    paese = table[day][i]
                    valuesa.append(row1.loc[:,get_column(dfd, paese).values].values[0])
                    dest2.append(table[day][i])
                    paese = table[day][i]
                    valuesb.append(row2.loc[:,get_column(dfd, paese).values].values[0])
            #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
            for k in valuesa:
                values1.append(k[np.argmin(k)])
            for k in valuesb:
                values2.append(k[np.argmin(k)])
            try:
                if( min(values1)>=min(values2) ):
                    curr1 = dest1[np.argmin(values1)]
                    d1 = d1 + values1[np.argmin(values1)]
                    seq_d1.append(d1)
                    #seq_h.append(math.ceil(min(values)))
                    sequence1.append(curr1)
                    paesi1.append(curr1)
                    #aggiorno la riga
                    row1 = dfd[get_row(dfd, curr1).values]
                else:
                    curr2 = dest2[np.argmin(values2)]
                    d2 = d2 + values2[np.argmin(values2)]
                    seq_h2.append(d2)
                    #seq_h.append(math.ceil(min(values)))
                    sequence2.append(curr2)
                    paesi2.append(curr2)
                    #aggiorno la riga
                    row2 = dfd[get_row(dfd, curr2).values]
            except:
                print(sequence1)
                print(sequence2)
        #dffurgoncino1[day]=table[day][f1].reset_index(drop=True)
        #dffurgoncino2[day]=table[day][f2].reset_index(drop=True)
        dffurgoncino1_km[day]=pd.Series(paesi1)
        dffurgoncino2_km[day]=pd.Series(paesi2)

In [18]:
dffurgoncino1_km.to_csv('../Dati/turni_f/furgoncino1_KM_routing.csv', index=False, sep=";", encoding = "ISO-8859-1")
dffurgoncino2_km.to_csv('../Dati/turni_f/furgoncino2_KM_routing.csv', index=False, sep=";", encoding = "ISO-8859-1")

In [19]:
dffurgoncino1_km.head()

,lunedi,martedi,mercoledi,giovedi,venerdi
0,sesto ed,"Cremona, Ugolani Dati, 4",Castelverde,"Cremona, sesto","crema, civerchi"
1,grumello,Persico Dosimo,Casalbuttano,"Cremona, Ugolani Dati, 4",Capergnanica
2,pizzighettone,Vescovato,Paderno,persico dosimo,bagnolo
3,MONTODINE,Ostiano,Soresina,grontardo,vaiano
4,CASTELLEONE,MANERBIO,Fiesco,pescarolo,monte cremasco


In [20]:
dffurgoncino1.head()

,lunedi,martedi,mercoledi,giovedi,venerdi
0,sesto ed,"Cremona, Ugolani Dati, 4",Castelverde,"Cremona, sesto","crema, civerchi"
1,grumello,Persico Dosimo,Annicco,"Cremona, Ugolani Dati, 4",Capergnanica
2,SORESINA,Ostiano,Castelleone,persico dosimo,bagnolo
3,TRIGOLO,San giovanni,Madignano,grontardo,vaiano
4,CASTELLEONE,"Scandolara Ravara, Italia",Crema,vescovato,monte cremasco
